In [8]:
import pandas as pd


In [13]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [14]:
from pyspark.sql import SparkSession

# Khởi tạo Spark session
spark = SparkSession.builder.appName("Music Recommendation System").getOrCreate()

# Đọc dữ liệu từ CSV
df = spark.read.csv("output.csv", header=True, sep=';', inferSchema=True)

# Hiển thị DataFrame
df.show(truncate=False)

Py4JError: An error occurred while calling None.org.apache.spark.sql.SparkSession. Trace:
py4j.Py4JException: Constructor org.apache.spark.sql.SparkSession([class org.apache.spark.SparkContext, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:834)



In [ ]:
#Lấy thông tin dòng số 50
row = df.collect()

In [ ]:
row[2000]

Row(Track ID='5zSKy9kDHd99HNPhiyB3bs', Track Name='El Valiente Cruz Vizcarra', Track Number='3', Duration (ms)=188368, Explicit='False', Album Name='Vino Maldito', Artists="['Luis y Julian']", Genres="['norteno']", Available Markets='AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY, CZ, DK, DO, DE, EC, EE, SV, FI, FR, GR, GT, HN, HK, HU, IS, IE, IT, LV, LT, LU, MY, MT, MX, NL, NZ, NI, NO, PA, PY, PE, PH, PL, PT, SG, SK, ES, SE, CH, TW, TR, UY, US, GB, AD, LI, MC, ID, JP, TH, VN, RO, IL, ZA, SA, AE, BH, QA, OM, KW, EG, MA, DZ, TN, LB, JO, PS, IN, BY, KZ, MD, UA, AL, BA, HR, ME, MK, RS, SI, KR, BD, PK, LK, GH, KE, NG, TZ, UG, AG, AM, BS, BB, BZ, BT, BW, BF, CV, CW, DM, FJ, GM, GE, GD, GW, GY, HT, JM, KI, LS, LR, MW, MV, ML, MH, FM, NA, NR, NE, PW, PG, PR, WS, SM, ST, SN, SC, SL, SB, KN, LC, VC, SR, TL, TO, TT, TV, VU, AZ, BN, BI, KH, CM, TD, KM, GQ, SZ, GA, GN, KG, LA, MO, MR, MN, NP, RW, TG, UZ, ZW, BJ, MG, MU, MZ, AO, CI, DJ, ZM, CD, CG, IQ, LY, TJ, VE, ET, XK', Popularity='0', Acousticn

In [ ]:
# Thống kê mô tả cho các cột số
df.describe().show()

+-------+--------------------+----------+--------------------+------------------+-----------------+----------+------------------------+--------------------+--------------------+--------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+
|summary|            Track ID|Track Name|        Track Number|     Duration (ms)|         Explicit|Album Name|                 Artists|              Genres|   Available Markets|          Popularity|      Acousticness|             Energy|   Instrumentalness|           Liveness|           Loudness|               Mode|        Speechiness|             Tempo|    Time Signature|            Valence|
+-------+--------------------+----------+--------------------+------------------+-----------------+----------+------------------------+--------------------+--------------------+--------------------+----------

#ML

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression  # Linear Regression
from pyspark.ml import Pipeline
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.evaluation import RegressionEvaluator

# Bước 1: Khởi tạo Spark session
spark = SparkSession.builder.appName("Popularity").getOrCreate()

# Bước 2: Tạo DataFrame mẫu
df = spark.read.csv("album_tracks_train_full.csv", header=True, sep=';', inferSchema=True)

# Chuyển đổi cột 'Popularity' thành kiểu IntegerType
df = df.withColumn("Popularity", df["Popularity"].cast(IntegerType()))

# Kiểm tra và xử lý giá trị null hoặc NaN trong cột 'Popularity'
df.select([count(when(isnan(c), c)).alias(c) for c in ["Popularity"]]).show()

# Loại bỏ các hàng có giá trị null hoặc NaN trong cột 'Popularity'
df = df.na.drop(subset=["Popularity"])

# Bước 3: Tiền xử lý dữ liệu
indexer_artists = StringIndexer(inputCol="Artists", outputCol="Artists_Index", handleInvalid="keep")
indexer_genres = StringIndexer(inputCol="Genres", outputCol="Genres_Index", handleInvalid="keep")

# Tạo VectorAssembler để kết hợp các đặc trưng
assembler = VectorAssembler(inputCols=["Artists_Index", "Genres_Index", "Acousticness", "Energy", "Instrumentalness", "Liveness", "Loudness", "Mode", "Speechiness", "Tempo", "Valence"], outputCol="features")

# Bước 4: Xây dựng mô hình sử dụng LinearRegression
linear_reg = LinearRegression(featuresCol="features", labelCol="Popularity")

# Tạo Pipeline
pipeline = Pipeline(stages=[indexer_artists, indexer_genres, assembler, linear_reg])

# Bước 5: Chia dữ liệu
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

# Huấn luyện mô hình
model = pipeline.fit(train_df)

# Dự đoán
predictions = model.transform(test_df)

# Bước 6: Tính toán các metric lỗi
evaluator = RegressionEvaluator(labelCol="Popularity", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)

evaluator.setMetricName("mse")
mse = evaluator.evaluate(predictions)

evaluator.setMetricName("r2")
r2 = evaluator.evaluate(predictions)

# Hiển thị kết quả
predictions = predictions.withColumn("prediction", col("prediction").cast(IntegerType()))
predictions.select("Track Name", "Artists", "Genres", "Popularity", "prediction").show()

# In các metric lỗi
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared: {r2}")

+----------+
|Popularity|
+----------+
|         0|
+----------+

+--------------------+--------------------+--------------------+----------+----------+
|          Track Name|             Artists|              Genres|Popularity|prediction|
+--------------------+--------------------+--------------------+----------+----------+
|Eu, Você, O Mar e...|    ['Luan Santana']|['sertanejo unive...|        65|        12|
|        Getting Away|  ['KR$NA', 'KR$NA']|['desi hip hop', ...|        14|        16|
|Suspension (Acous...|          ['Lights']|['indie poptimism...|        17|        14|
|AOGG (feat. Tay K...|['Kenny Muney', '...| ['memphis hip hop']|        24|        16|
|             Disgust|  ['KMFDM', 'KMFDM']|['industrial meta...|        13|        11|
|     Paradise Is You|         ['La Roux']|['electropop', 'a...|        26|        20|
| Compay Póngase Duro|  ['Lalo Rodriguez']|['salsa', 'salsa ...|         9|        14|
|6 Épigraphes anti...|['Claude Debussy'...|['impressionism',...|

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from xgboost.spark import SparkXGBRegressor  # Nhập XGBoostRegressor
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, when, count

# Bước 1: Khởi tạo Spark session
spark = SparkSession.builder.appName("Popularity").getOrCreate()

# Bước 2: Tạo DataFrame từ file CSV
df = spark.read.csv("album_tracks_train_full.csv", header=True, sep=';', inferSchema=True)

# Bước 3: Chuyển đổi cột 'Popularity' thành kiểu IntegerType
df = df.withColumn("Popularity", df["Popularity"].cast(IntegerType()))

# Bước 4: Kiểm tra và xử lý giá trị null hoặc NaN trong cột 'Popularity'
df.select([count(when(isnan(c), c)).alias(c) for c in ["Popularity"]]).show()
df = df.na.drop(subset=["Popularity"])  # Loại bỏ các hàng có giá trị null hoặc NaN

# Bước 5: Tiền xử lý dữ liệu
# Chuyển đổi các cột danh mục thành số
indexer_artists = StringIndexer(inputCol="Artists", outputCol="Artists_Index", handleInvalid="keep")
indexer_genres = StringIndexer(inputCol="Genres", outputCol="Genres_Index", handleInvalid="keep")

# Tạo VectorAssembler để kết hợp các đặc trưng
assembler = VectorAssembler(inputCols=["Artists_Index", "Genres_Index", "Acousticness", "Energy",
                                        "Instrumentalness", "Liveness", "Loudness",
                                        "Mode", "Speechiness", "Tempo", "Time Signature", "Valence"],
                            outputCol="features")

# Bước 6: Chuẩn hóa các đặc trưng số
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)

# Bước 7: Sử dụng XGBoostRegressor
xgboost_reg = SparkXGBRegressor(features_col="scaled_features", label_col="Popularity")

# Bước 8: Tạo Pipeline bao gồm tất cả các bước
pipeline = Pipeline(stages=[indexer_artists, indexer_genres, assembler, scaler, xgboost_reg])

# Bước 9: Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

# Bước 10: Tuning hyperparameters với CrossValidator
paramGrid = ParamGridBuilder() \
    .addGrid(xgboost_reg.max_depth, [3, 6, 10]) \
    .addGrid(xgboost_reg.learning_rate, [0.1, 0.01, 0.001]) \
    .addGrid(xgboost_reg.reg_lambda, [1, 10]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol="Popularity", predictionCol="prediction", metricName="mae"),
                          numFolds=3)  # 3-fold CrossValidation

# Bước 11: Huấn luyện mô hình
cv_model = crossval.fit(train_df)

# Bước 12: Dự đoán trên tập kiểm tra
predictions = cv_model.transform(test_df)

# Bước 13: Tính toán các metric lỗi
evaluator = RegressionEvaluator(labelCol="Popularity", predictionCol="prediction")
mae = evaluator.evaluate(predictions)
mse = evaluator.setMetricName("mse").evaluate(predictions)
r2 = evaluator.setMetricName("r2").evaluate(predictions)

# Bước 14: Hiển thị kết quả dự đoán
predictions.select("Track Name", "Artists", "Genres", "Popularity", "prediction").show()

# Bước 15: In các metric lỗi
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared: {r2}")

# Dừng Spark session
spark.stop()

+----------+
|Popularity|
+----------+
|         0|
+----------+



INFO:XGBoost-PySpark:Running xgboost-2.1.3 on 1 workers with
	booster params: {'device': 'cpu', 'learning_rate': 0.1, 'max_depth': 3, 'objective': 'reg:squarederror', 'reg_lambda': 1, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!
INFO:XGBoost-PySpark:Running xgboost-2.1.3 on 1 workers with
	booster params: {'device': 'cpu', 'learning_rate': 0.1, 'max_depth': 3, 'objective': 'reg:squarederror', 'reg_lambda': 10, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!
INFO:XGBoost-PySpark:Running xgboost-2.1.3 on 1 workers with
	booster params: {'device': 'cpu', 'learning_rate': 0.01, 'max_depth': 3, 'objective': 'reg:squarederror', 'reg_lambda': 1, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_rou

+--------------------+--------------------+--------------------+----------+------------------+
|          Track Name|             Artists|              Genres|Popularity|        prediction|
+--------------------+--------------------+--------------------+----------+------------------+
|Eu, Você, O Mar e...|    ['Luan Santana']|['sertanejo unive...|        65|19.651870727539062|
|        Getting Away|  ['KR$NA', 'KR$NA']|['desi hip hop', ...|        14|21.586992263793945|
|Suspension (Acous...|          ['Lights']|['indie poptimism...|        17|16.666772842407227|
|AOGG (feat. Tay K...|['Kenny Muney', '...| ['memphis hip hop']|        24| 21.61313247680664|
|             Disgust|  ['KMFDM', 'KMFDM']|['industrial meta...|        13| 12.55388355255127|
|     Paradise Is You|         ['La Roux']|['electropop', 'a...|        26|26.241558074951172|
| Compay Póngase Duro|  ['Lalo Rodriguez']|['salsa', 'salsa ...|         9| 14.90137767791748|
|6 Épigraphes anti...|['Claude Debussy'...|['impre